## The Battle of Neighborhoods (Week 2)

**Exploring the data:**

In [1]:
import pandas as pd
import numpy as np
import requests

**Finding center of cities:**

In [2]:
# ! pip install folium==0.5.0

In [3]:
from geopy.geocoders import Nominatim
import folium

In [4]:
cities_list = []

cities = ['New York, NY', 'Philadelphia, PA', 'Boston, MA', 'Baltimore, MD', 'Pittsburgh, PA']

for city in cities:
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    cities_list.append([city.split(',')[0].upper(),[latitude, longitude]])
print(cities_list)

[['NEW YORK', [40.7127281, -74.0060152]], ['PHILADELPHIA', [39.9527237, -75.1635262]], ['BOSTON', [42.3602534, -71.0582912]], ['BALTIMORE', [39.2908816, -76.610759]], ['PITTSBURGH', [40.4416941, -79.9900861]]]


**Setting up variables for query:**

In [5]:
CLIENT_ID = 'WUPUAH5PA54ZBTSDADZUGWT0ROUITGR5S1ZPV3EIV1DYIVP3'
CLIENT_SECRET = 'F0VENJWDIYB3JQYW3DOON402F0DK34O2MSR42145SCXS0UYM'
VERSION = '20201101'
LIMIT = 30
RADIUS = 8046 # meters
SEARCH_QUERY = 'Pizza'

In [6]:
for x in cities_list:
    LAT = x[1][0]
    LNG = x[1][1]
#     print(LAT,LNG)

In [18]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,LAT,LNG,SEARCH_QUERY,RADIUS,LIMIT)

In [22]:
results = requests.get(url).json()

In [23]:
venues = results['response']['venues']
dataframe = pd.json_normalize(venues)

**Grabbing data from Foursquare API:**

In [21]:
columns = ['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.address', 'location.lat', 'location.lng', 'location.labeledLatLngs', 'location.distance', 'location.postalCode', 'location.cc', 'location.city', 'location.state', 'location.country', 'location.formattedAddress', 'delivery.id', 'delivery.url', 'delivery.provider.name', 'delivery.provider.icon.prefix', 'delivery.provider.icon.sizes', 'delivery.provider.icon.name', 'venuePage.id', 'location.neighborhood', 'location.crossStreet']
df = pd.DataFrame(columns=columns)
for x in cities_list:
    LAT = x[1][0]
    LNG = x[1][1]
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,LAT,LNG,SEARCH_QUERY,RADIUS,LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    df2 = pd.json_normalize(venues)
    df = df.append(df2, ignore_index=True)

In [24]:
df.head(3)

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id,location.neighborhood,location.crossStreet
0,4f045eeb00399761c77301e3,Prince Street Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1604267677,False,27 Prince St,40.723093,-73.994527,"[{'label': 'display', 'lat': 40.72309326145674...",1506,...,"[27 Prince St (btwn Mott & Elizabeth St), New ...",264357,https://www.seamless.com/menu/prince-st-pizza-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,99329256,NaN,btwn Mott & Elizabeth St
1,4a4560a7f964a5201aa81fe3,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1604267677,False,2 Gold St,40.707634,-74.006996,"[{'label': 'display', 'lat': 40.7076335455641,...",573,...,"[2 Gold St (at Maiden Ln.), New York, NY 10038...",NaN,NaN,NaN,NaN,NaN,NaN,77559825,NaN,at Maiden Ln.
2,4af216e7f964a520d0e521e3,Dona Bella Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1604267677,False,154 Church Steet,40.714990,-74.007850,"[{'label': 'display', 'lat': 40.71499001654046...",295,...,"[154 Church Steet (Chambers St), New York, NY ...",69858,https://www.seamless.com/menu/little-italy-piz...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN,Chambers St


**Cleaning dataframe:**

In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
dataframe_filtered = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [26]:
dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Prince Street Pizza,Pizza Place,27 Prince St,40.723093,-73.994527,"[{'label': 'display', 'lat': 40.72309326145674...",1506,10012,US,New York,NY,United States,"[27 Prince St (btwn Mott & Elizabeth St), New ...",NaN,btwn Mott & Elizabeth St,4f045eeb00399761c77301e3
1,Harry's Italian Pizza Bar,Italian Restaurant,2 Gold St,40.707634,-74.006996,"[{'label': 'display', 'lat': 40.7076335455641,...",573,10038,US,New York,NY,United States,"[2 Gold St (at Maiden Ln.), New York, NY 10038...",NaN,at Maiden Ln.,4a4560a7f964a5201aa81fe3
2,Dona Bella Pizza,Pizza Place,154 Church Steet,40.714990,-74.007850,"[{'label': 'display', 'lat': 40.71499001654046...",295,10007,US,New York,NY,United States,"[154 Church Steet (Chambers St), New York, NY ...",NaN,Chambers St,4af216e7f964a520d0e521e3
3,Little Italy Gourmet Pizza,Pizza Place,11 Park Pl,40.713044,-74.008334,"[{'label': 'display', 'lat': 40.713044, 'lng':...",198,10007,US,New York,NY,United States,"[11 Park Pl (Broadway and Church), New York, N...",NaN,Broadway and Church,4bf6ae6c94b2a59392deadee
4,Stage Door Pizza,Pizza Place,26 Vesey St,40.711838,-74.010189,"[{'label': 'display', 'lat': 40.71183761933663...",365,10007,US,New York,NY,United States,"[26 Vesey St, New York, NY 10007, United States]",NaN,NaN,52698af411d29a41c427892c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Pizza Parma,Pizza Place,823 Penn Ave,40.443886,-79.998573,"[{'label': 'display', 'lat': 40.44388580322265...",759,15222,US,Pittsburgh,PA,United States,"[823 Penn Ave, Pittsburgh, PA 15222, United St...",NaN,NaN,59f206191bc7045cb0fdef99
146,Pizza Vespucci,Pizza Place,1710 Forbes Ave,40.437377,-79.982265,"[{'label': 'display', 'lat': 40.43737700000000...",818,15219,US,Pittsburgh,PA,United States,"[1710 Forbes Ave, Pittsburgh, PA 15219, United...",NaN,NaN,51796804498e66dca6f16124
147,Pizza Hut,Pizza Place,430 Wood St,40.440151,-80.000542,"[{'label': 'display', 'lat': 40.44015121459961...",902,15222,US,Pittsburgh,PA,United States,"[430 Wood St, Pittsburgh, PA 15222, United Sta...",NaN,NaN,4f32500719836c91c7cb5cde
148,brother's pizza,None,Benner pike,40.440394,-79.995415,"[{'label': 'display', 'lat': 40.44039429610388...",474,NaN,US,Pittsburgh,PA,United States,"[Benner pike, Pittsburgh, PA, United States]",NaN,NaN,4b2c4201f964a520c3c424e3
